<a href="https://colab.research.google.com/github/vkaramanis/Coursera_Capstone/blob/main/Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

# Task 1

Fetch Data from wiki

In [2]:
headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')

Create table

In [3]:
table = soup.find("table")

parsed_data = []
for row in table.findAll('tr'):
  for td in row.findAll('td'):
    data = td.get_text(separator=" ", strip=True).split("(")
    po = np.nan
    borough = np.nan
    neighbourhood = np.nan

    for idx, d in enumerate(data):
      if idx == 0:
        po = d[:3]
        borough = np.nan if re.search('Not assigned',d[3:]) else d[3:]
      elif idx == 1:
        neighbourhood = re.sub(r' / ', ', ', d)
        neighbourhood = re.sub(r'\)*', '', neighbourhood)
      else:
        clean = re.sub(r' / ', ', ', d)
        clean = re.sub(r'\)*', '', clean)
        neighbourhood += ', ' + clean
    parsed_data += [[po, borough,neighbourhood]]

df = pd.DataFrame(parsed_data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df.set_index('PostalCode', inplace=True)
df

,Borough,Neighbourhood
PostalCode,,
M1A,NaN,NaN
M2A,NaN,NaN
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...
M5Z,NaN,NaN
M6Z,NaN,NaN
M7Z,NaN,NaN


Drop null Boroughs

In [4]:
df = df[df['Borough'].notnull()]
df.head()

,Borough,Neighbourhood
PostalCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Queen's Park,Ontario Provincial Government


There are no null Neighbourhoods

In [5]:
df['Neighbourhood'].isnull().unique()

array([False])

Shape of dataframe

In [6]:
df.shape

(103, 2)

# Task 2

In [9]:
!pip install geocoder
import geocoder

Limit error so i used the provided dataset

In [ ]:
latitude = []
longitude = []

for row in df.iterrows():
  lat_lng_coords = None
  while lat_lng_coords is None:
    g = geocoder.google('{}, Toronto, Ontario'.format(row[0]))
    lat_lng_coords = g.latlng

  latitude = lat_lng_coords[0]
  longitude = lat_lng_coords[1]


Fetch the dataset

In [21]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv

--2021-07-27 10:54:21--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2788 (2.7K) [text/csv]
Saving to: ‘Geospatial_Coordinates.csv’

Geospatial_Coordina 100%[===================>]   2.72K  --.-KB/s    in 0s      

2021-07-27 10:54:21 (361 MB/s) - ‘Geospatial_Coordinates.csv’ saved [2788/2788]



Join it with our dataset

In [28]:
coordinates = pd.read_csv('/content/Geospatial_Coordinates.csv')

df = df.merge(coordinates, left_on='PostalCode', right_on='Postal Code')
df = df.drop('Postal Code', 1)
df.head()

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
